# Part 7a - Youth Interaction with Opportunity Provider

## Also open in another tab/window the [Opportunity Provider Notebook](http://localhost:8888/notebooks/Part%207b%20-%20Youth%20and%20Opportunity%20Provider%20interaction%20-%20Opportunity%20Provider.ipynb)

## 1. Initialise a controller for Yoma Youth Sub Wallet 1

In [1]:
%autoawait
import time
import asyncio
from termcolor import colored,cprint

from aries_basic_controller.aries_controller import AriesAgentController
    
WEBHOOK_HOST = "0.0.0.0"
WEBHOOK_PORT = 8022
WEBHOOK_BASE = ""
ADMIN_URL = "http://yoma-multi-tenant-agent:8021"

# JWT KEY for Youth 1 Sub Wallet
%store -r youth1_token

# Based on the aca-py agent you wish to control
agent_controller = AriesAgentController(webhook_host=WEBHOOK_HOST, webhook_port=WEBHOOK_PORT,
                                       webhook_base=WEBHOOK_BASE, admin_url=ADMIN_URL, jwt_token=youth1_token)

Unclosed client session
client_session: <aiohttp.client.ClientSession object at 0x7f84c8740be0>


IPython autoawait is `on`, and set to use `asyncio`


## 2. Register Listeners

The handler should get called every time the controller receives a webhook with the topic issue_credential, printing out the payload. The agent calls to this webhook every time it receives an issue-credential protocol message from a credential.

In [5]:
loop = asyncio.get_event_loop()
loop.create_task(agent_controller.listen_webhooks())

def cred_handler(payload):
    print("Handle Credentials")
    exchange_id = payload['credential_exchange_id']
    state = payload['state']
    role = payload['role']
    attributes = payload['credential_proposal_dict']['credential_proposal']['attributes']
    print(f"Credential exchange {exchange_id}, role: {role}, state: {state}")
    print(f"Offering: {attributes}")
    
cred_listener = {
    "topic": "issue_credential",
    "handler": cred_handler
}

def connections_handler(payload):
    global STATE
    connection_id = payload["connection_id"]
    print("Connection message", payload, connection_id)
    STATE = payload['state']
    if STATE == 'active':
#         print('Connection {0} changed state to active'.format(connection_id))
        print(colored("Connection {0} changed state to active".format(connection_id), "red", attrs=["bold"]))


connection_listener = {
    "handler": connections_handler,
    "topic": "connections"
}

agent_controller.register_listeners([cred_listener,connection_listener], defaults=True)


Task exception was never retrieved
future: <Task finished name='Task-6' coro=<AriesAgentController.listen_webhooks() done, defined at /aries_basic_controller/aries_controller.py:140> exception=OSError(98, "error while attempting to bind on address ('0.0.0.0', 8022): address already in use")>
Traceback (most recent call last):
  File "/aries_basic_controller/aries_controller.py", line 147, in listen_webhooks
    await self.webhook_site.start()
  File "/opt/conda/lib/python3.8/site-packages/aiohttp/web_runner.py", line 121, in start
    self._server = await loop.create_server(
  File "/opt/conda/lib/python3.8/asyncio/base_events.py", line 1463, in create_server
    raise OSError(err.errno, 'error while attempting '
OSError: [Errno 98] error while attempting to bind on address ('0.0.0.0', 8022): address already in use


## 8a. Check for any existing connections

In [6]:
# Check for existing connections
connection = await agent_controller.connections.get_connections()
print("EXISTING CONNECTIONS")
for key, value in connection.items():
    for item in value:
        print('ConnectionID:', item['connection_id'], 'Status:',item['state'])

EXISTING CONNECTIONS
ConnectionID: 98e7a2a3-80d9-4dad-9efa-136e034f4982 Status: active
ConnectionID: 66a23385-4466-48c4-aa2d-4eb9878fe29d Status: active


### 8. Paste the invitation from the Yoma Opportunity Provider notebook into the invitation variable here

In [3]:
invitation = {'@type': 'did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/connections/1.0/invitation', '@id': '4ad97961-4779-4f38-a84e-95f43cf298bb', 'serviceEndpoint': 'http://677011df58cb.ngrok.io', 'recipientKeys': ['BRGR2YLXufLQB9jysudMiYP9HASbPVBdW4b1zx2zLPmY'], 'imageUrl': 'https://aries.ca/images/sample.png', 'label': 'Yoma Opportunity Provider 1'}

### 9. Accept the invitation, then move to [Yoma Opportunity Provider Issuer notebook](http://localhost:8888/notebooks/Part%207b%20-%20Youth%20and%20Opportunity%20Provider%20interaction%20-%20Opportunity%20Provider.ipynb)

In [4]:
# Receive Invitation
response = await agent_controller.connections.accept_connection(invitation)
# Print out accepted Invite and Alice's connection ID
print("Connection", response)
connection_id = response["connection_id"]
STATE = response["state"]

Connection {'created_at': '2021-03-16 15:38:34.048595Z', 'invitation_mode': 'once', 'connection_id': '66a23385-4466-48c4-aa2d-4eb9878fe29d', 'routing_state': 'none', 'updated_at': '2021-03-16 15:38:34.085660Z', 'my_did': 'Nm2Vg3KH2FMZeicGwyZese', 'state': 'request', 'invitation_key': 'BRGR2YLXufLQB9jysudMiYP9HASbPVBdW4b1zx2zLPmY', 'request_id': 'ef991347-e2e4-4852-ab4c-bb612e752561', 'accept': 'manual', 'their_role': 'inviter', 'their_label': 'Yoma Opportunity Provider 1', 'rfc23_state': 'request-sent'}


### 13. Check if connection state is active

It should be if you send a trust ping from Alice's side

In [7]:
# Print connection list
connection = await agent_controller.connections.get_connection(connection_id)
print("Youth Wallet 1 AGENT CONNECTION")
print(connection)
print("State:", connection["state"])

Youth Wallet 1 AGENT CONNECTION
{'created_at': '2021-03-16 15:38:34.048595Z', 'invitation_mode': 'once', 'connection_id': '66a23385-4466-48c4-aa2d-4eb9878fe29d', 'routing_state': 'none', 'updated_at': '2021-03-16 15:38:36.807394Z', 'my_did': 'Nm2Vg3KH2FMZeicGwyZese', 'state': 'active', 'invitation_key': 'BRGR2YLXufLQB9jysudMiYP9HASbPVBdW4b1zx2zLPmY', 'request_id': 'ef991347-e2e4-4852-ab4c-bb612e752561', 'accept': 'manual', 'their_role': 'inviter', 'their_label': 'Yoma Opportunity Provider 1', 'rfc23_state': 'completed', 'their_did': 'JpGtE9KmwdX3pLRR2KKUy7'}
State: active


## 10. Check Credential Exchange Records

The agent will have at least one record if you have run through the issuer notebook up until send credential.


In [13]:
response = await agent_controller.issuer.get_records()
results = response["results"]
if len(results) == 0:
    print("You need to first send a credential from the issuer notebook (Alice)")
else:
    cred_record = results[0]
    cred_ex_id = cred_record['credential_exchange_id']
    state = cred_record['state']
    role = cred_record['role']
    attributes = results[0]['credential_proposal_dict']['credential_proposal']['attributes']
    print(f"Credential exchange {cred_ex_id}, role: {role}, state: {state}")
    print(f"Being offered: {attributes}")



Credential exchange a812cbaf-ed13-4d04-9dfd-8ced099172e4, role: holder, state: credential_acked
Being offered: [{'name': 'fullname', 'value': 'Lohan Spies'}, {'name': 'mobile', 'value': '0832582698'}, {'name': 'email', 'value': 'eax@eax.co.za'}]


## 12. Store the credential

Once the issuer has responded to a request by sending the credential, the holder needs to store it to save the credential for later.

First check that the credential record is in the credential_received state


In [9]:
record = await agent_controller.issuer.get_record_by_id(cred_ex_id)
state = record['state']
role = record['role']
print(f"Credential exchange {cred_ex_id}, role: {role}, state: {state}")

Credential exchange a812cbaf-ed13-4d04-9dfd-8ced099172e4, role: holder, state: credential_acked


## 11. End of Tutorial

Be sure to terminate the controller so you can run another tutorial.

In [13]:
response = await agent_controller.terminate()
print(response)

None


# Proceed to Part 4 on [Verifier Notebook](http://localhost:8889/notebooks/Part%204%20-%20Verify%20a%20Presentation.ipynb)

Here you will present the attributes issued to you within this tutorial to a verifying entity.